In [1]:
import cv2
from PIL import Image
import numpy as np

import pandas as pd

from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam,RMSprop,SGD
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(64,64,1)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
#model.add(Conv2D(32, (3, 3)))
#model.add(BatchNormalization(axis=-1))
#model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
#model.add(Conv2D(64, (3, 3)))
#model.add(BatchNormalization(axis=-1))
#model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#-------------------

#model.add(Conv2D(128,(3, 3)))
#model.add(BatchNormalization(axis=-1))
#model.add(Activation('relu'))
#model.add(Conv2D(128, (3, 3)))
#model.add(BatchNormalization(axis=-1))
#model.add(Activation('relu'))
#model.add(Conv2D(128, (3, 3)))
#model.add(BatchNormalization(axis=-1))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2,2)))

# #--------------------

# model.add(Conv2D(256,(3, 3)))
# model.add(BatchNormalization(axis=-1))
# model.add(Activation('relu'))
# model.add(Conv2D(256, (3, 3)))
# model.add(BatchNormalization(axis=-1))
# model.add(Activation('relu'))
# model.add(Conv2D(256, (3, 3)))
# model.add(BatchNormalization(axis=-1))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))

#--------------------------------

# model.add(Conv2D(512,(3, 3)))
# model.add(BatchNormalization(axis=-1))
# model.add(Activation('relu'))
# model.add(Conv2D(512, (3, 3)))
# model.add(BatchNormalization(axis=-1))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

# Fully connected layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(17))

model.add(Activation('softmax'))

In [2]:

model = Sequential()

model.add(Conv2D(32,kernel_size=3,activation='relu',input_shape=(64,64,1)))
model.add(BatchNormalization())
#model.add(Conv2D(32,kernel_size=3,activation='relu'))
#model.add(BatchNormalization())
model.add(Conv2D(32,kernel_size=5,strides=2,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(64,kernel_size=3,activation='relu'))
model.add(BatchNormalization())
#model.add(Conv2D(64,kernel_size=3,activation='relu'))
#model.add(BatchNormalization())
model.add(Conv2D(64,kernel_size=5,strides=2,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(128,kernel_size=3,activation='relu'))
model.add(BatchNormalization())
#model.add(Conv2D(128,kernel_size=3,activation='relu'))
#model.add(BatchNormalization())
model.add(Conv2D(128,kernel_size=5,strides=2,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

# model.add(Conv2D(256,kernel_size=3,activation='relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(256,kernel_size=3,activation='relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(256,kernel_size=5,strides=2,padding='same',activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.25))

# model.add(Conv2D(512,kernel_size=3,activation='relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(512,kernel_size=3,activation='relu'))
# model.add(BatchNormalization())
# model.add(Conv2D(512,kernel_size=5,strides=2,padding='same',activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))


In [3]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

#Adding Callbacks and learning rate reductions
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=1, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [4]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [6]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        shear_range=0.2,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True)
valid_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.2,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        './train',  # this is the target directory
        target_size=(64,64),  # all images will be resized to 150x150
        batch_size=64,
        color_mode='grayscale',
        class_mode='categorical')  
valid_generator = valid_datagen.flow_from_directory(
    directory='./test',
    target_size=(64, 64),
    color_mode="grayscale",
    batch_size=64,
    class_mode="categorical"
)

Found 8750 images belonging to 5 classes.
Found 1250 images belonging to 5 classes.


In [61]:
print(trai)

(array([[[[0.0000000e+00],
         [0.0000000e+00],
         [0.0000000e+00],
         ...,
         [0.0000000e+00],
         [0.0000000e+00],
         [0.0000000e+00]],

        [[0.0000000e+00],
         [0.0000000e+00],
         [0.0000000e+00],
         ...,
         [0.0000000e+00],
         [0.0000000e+00],
         [0.0000000e+00]],

        [[0.0000000e+00],
         [0.0000000e+00],
         [0.0000000e+00],
         ...,
         [0.0000000e+00],
         [0.0000000e+00],
         [0.0000000e+00]],

        ...,

        [[0.0000000e+00],
         [0.0000000e+00],
         [0.0000000e+00],
         ...,
         [6.9872658e-03],
         [5.7945652e-03],
         [2.8901217e-03]],

        [[0.0000000e+00],
         [0.0000000e+00],
         [0.0000000e+00],
         ...,
         [0.0000000e+00],
         [2.5970088e-03],
         [5.5014524e-03]],

        [[0.0000000e+00],
         [0.0000000e+00],
         [0.0000000e+00],
         ...,
         [0.0000000e+00],
       

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=8750//64,validation_data=valid_generator,validation_steps=1250//64, epochs=1, callbacks=[learning_rate_reduction])

Epoch 1/1
  5/136 [>.............................] - ETA: 4:02 - loss: 0.0203 - acc: 0.9938

In [9]:
model.save('./model5.h5')

In [59]:
test = []

for i in range(4,5):
    image=cv2.imread(str(i)+'.jpg')
    image = cv2.resize(image, (28,28), interpolation = cv2.INTER_AREA)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    ret, thresh = cv2.threshold(blur, 250, 255, cv2.THRESH_BINARY_INV)
    cv2.imshow("fekbkj",thresh)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    test.append(thresh)
test = np.array(test)
test = test.astype(np.float16)
test /= 255

error: OpenCV(3.4.2) /io/opencv/modules/imgproc/src/resize.cpp:4044: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [57]:
model1 = load_model('model4.h5')

In [58]:
test = test.reshape((1,28,28,1))

prediction1 = model1.predict_classes(test)
print(prediction1)

[13]
